# Spatial Queries

In spatial analysis, our goal is not just to make nice maps, but to actually run analyses that leverage the explicitly spatial nature of our data. The process of doing this is known as **spatial analysis**.

To construct spatial analyses, we string together series of spatial operations in such a way that the end result answers our question of interest. There are many such spatial operations. These are known as **spatial queries**.

These queries can be divided into:

- **Measurement queries**
    - What is feature A's **length**?
    - What is feature A's **area**?
    - What is feature A's **perimeter**?
    - What is feature A's **distance** from feature B?
- **Relationship queries**
    - Is feature A **within** feature B?
    - Does feature A **intersect** with feature B?
    - Does feature A **cross** feature B?
    
Spatial queries are not limited to the examples we've shown here.

We'll work through examples of each of those types of queries. Then, we'll see an example of a very common spatial analysis that is a conceptual amalgam of those two types: **proximity analysis**.

<!--
- Expected time to complete
    - Lecture + Questions: 45 minutes
    - Exercises: 20 minutes
-->

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pyproj

%matplotlib inline  

## Load and Prep Some Data

Let's read in our Census tracts data again:

In [2]:
example = gpd.read_file("/Users/aptegroup/Downloads/Thomas-Research/example.shp")

In [3]:
example.head()

,I_CELL,J_CELL,NH3,NOX,PM25,ROG,SOX,HEIGHT_M,geometry
0,369,934,0.000009,0.052552,0.000867,0.112752,0.000100,0.0,"POLYGON ((-316000.000 369000.000, -316000.000 ..."
1,370,934,0.000065,0.363577,0.005995,0.780061,0.000694,0.0,"POLYGON ((-315000.000 369000.000, -315000.000 ..."
2,370,935,0.000002,0.008569,0.000141,0.018384,0.000016,0.0,"POLYGON ((-315000.000 370000.000, -315000.000 ..."
3,376,958,0.018784,1.126114,0.028532,1.080482,0.005048,0.0,"POLYGON ((-309000.000 393000.000, -309000.000 ..."
4,376,959,0.000169,0.575581,0.011609,1.164903,0.001338,0.0,"POLYGON ((-309000.000 394000.000, -309000.000 ..."


Then, we'll grab just the Alameda Country tracts.

In [4]:
example['geometry'].head()

0    POLYGON ((-316000.000 369000.000, -316000.000 ...
1    POLYGON ((-315000.000 369000.000, -315000.000 ...
2    POLYGON ((-315000.000 370000.000, -315000.000 ...
3    POLYGON ((-309000.000 393000.000, -309000.000 ...
4    POLYGON ((-309000.000 394000.000, -309000.000 ...
Name: geometry, dtype: geometry

In [5]:
sheet = gpd.read_file("/Users/aptegroup/Downloads/Thomas-Research/stk12/stk12_splice - stk12_FULL_0308.csv")


In [6]:
sheet.Long_WGS84

0    -117.383
1    -117.383
2    -117.383
3    -117.383
4    -117.383
5    -117.383
Name: Long_WGS84, dtype: object

In [7]:
sheet_gdf = gpd.GeoDataFrame(
                sheet, geometry = gpd.points_from_xy(sheet.Long_WGS84,sheet.Lat_WGS84), crs='EPSG:4326')
sheet_gdf.head()

,field_1,arbid_facid,ARBID,facid2012,facid2017,facidother,arbid_facid_12,RECODE,DIS,AB,...,VOC,NH3,Lat_WGS84,Long_WGS84,DUP,count_ARB,TON,NA_val2,LL_SOURCE,geometry
0,1,100001_50001051,100001,50001051,50001051,NA,100001_50001051,FALSE,MOJ,MD,...,0.0508,7.0234,35.766,-117.383,FALSE,1,FALSE,FALSE,LL,POINT (-117.38300 35.76600)
1,2,100001_50001051,100001,50001051,50001051,NA,100001_50001051,FALSE,MOJ,MD,...,0,0,35.766,-117.383,FALSE,1,FALSE,FALSE,LL,POINT (-117.38300 35.76600)
2,3,100001_50001051,100001,50001051,50001051,NA,100001_50001051,FALSE,MOJ,MD,...,0,0,35.766,-117.383,FALSE,1,FALSE,FALSE,LL,POINT (-117.38300 35.76600)
3,4,100001_50001051,100001,50001051,50001051,NA,100001_50001051,FALSE,MOJ,MD,...,0,0,35.766,-117.383,FALSE,1,FALSE,FALSE,LL,POINT (-117.38300 35.76600)
4,5,100001_50001051,100001,50001051,50001051,NA,100001_50001051,FALSE,MOJ,MD,...,0,0,35.766,-117.383,FALSE,1,FALSE,FALSE,LL,POINT (-117.38300 35.76600)


In [8]:
column_change = sheet.rename(columns={"arbid_facid": "I_CELL", "facid2017": "J_CELL"})
column_change.head()

,field_1,I_CELL,ARBID,facid2012,J_CELL,facidother,arbid_facid_12,RECODE,DIS,AB,...,VOC,NH3,Lat_WGS84,Long_WGS84,DUP,count_ARB,TON,NA_val2,LL_SOURCE,geometry
0,1,100001_50001051,100001,50001051,50001051,NA,100001_50001051,FALSE,MOJ,MD,...,0.0508,7.0234,35.766,-117.383,FALSE,1,FALSE,FALSE,LL,POINT (-117.38300 35.76600)
1,2,100001_50001051,100001,50001051,50001051,NA,100001_50001051,FALSE,MOJ,MD,...,0,0,35.766,-117.383,FALSE,1,FALSE,FALSE,LL,POINT (-117.38300 35.76600)
2,3,100001_50001051,100001,50001051,50001051,NA,100001_50001051,FALSE,MOJ,MD,...,0,0,35.766,-117.383,FALSE,1,FALSE,FALSE,LL,POINT (-117.38300 35.76600)
3,4,100001_50001051,100001,50001051,50001051,NA,100001_50001051,FALSE,MOJ,MD,...,0,0,35.766,-117.383,FALSE,1,FALSE,FALSE,LL,POINT (-117.38300 35.76600)
4,5,100001_50001051,100001,50001051,50001051,NA,100001_50001051,FALSE,MOJ,MD,...,0,0,35.766,-117.383,FALSE,1,FALSE,FALSE,LL,POINT (-117.38300 35.76600)


In [15]:
em_only = column_change[["I_CELL", "J_CELL", "NH3", "NOX", "PM2.5", "SOX", "STKHT","VOC", "geometry"]]
em_only.head()
#Ask what ROG means, source height vs stack height,

,I_CELL,J_CELL,NH3,NOX,PM2.5,SOX,STKHT,VOC,geometry
0,100001_50001051,50001051,7.0234,277.8495,29.6655,197.6732,260,0.0508,POINT (-117.38300 35.76600)
1,100001_50001051,50001051,0,0,12.5701,0,80,0,POINT (-117.38300 35.76600)
2,100001_50001051,50001051,0,0,0.0194,0,90,0,POINT (-117.38300 35.76600)
3,100001_50001051,50001051,0,0,0.121,0,130,0,POINT (-117.38300 35.76600)
4,100001_50001051,50001051,0,0,0.0093,0,15,0,POINT (-117.38300 35.76600)


In [10]:
example.columns

Index(['I_CELL', 'J_CELL', 'NH3', 'NOX', 'PM25', 'ROG', 'SOX', 'HEIGHT_M',
       'geometry'],
      dtype='object')

In [11]:
example.crs

<Derived Projected CRS: PROJCS["MM5_sphere",GEOGCS["Unknown datum based up ...>
Name: MM5_sphere
Axis Info [cartesian]:
- [east]: Easting (metre)
- [north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Lambert Conic Conformal (2SP)
Datum: D_custom
- Ellipsoid: custom
- Prime Meridian: custom

In [13]:
em_only.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [16]:
em_only.to_file("basic.shp")

In [25]:
swap = em_only.to_crs("epsg:4326")

In [26]:
swap.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

## Measurement Queries

We'll start off with some simple measurement queries.

For example, here's how we can get the areas of each of our Census tracts:

In [ ]:
census_tracts_ac.area

Okay! We got...numbers?

What do those numbers mean? What are our units? And if we're not sure, how might be find out?

Let's take a look at our CRS:

In [ ]:
census_tracts_ac.crs

Ah-ha! We're working in an unprojected CRS, with units of decimal degrees.

**When doing spatial analysis, we will almost always want to work in a projected CRS that has natural distance units, such as meters!**

Time to project!

As previously, we'll use UTM Zone 10N with a NAD83 data. This is a good choice for our region of interest.

In [ ]:
census_tracts_ac_utm10 = census_tracts_ac.to_crs("epsg:26910")

In [ ]:
census_tracts_ac_utm10.crs

Now, let's try our area calculation again.

In [ ]:
census_tracts_ac_utm10.area

That looks much more reasonable! What are our units, now?

You may have noticed that our Census tracts already have an area column in them.

Let's do a confidence check on our results.

In [ ]:
# Calculate the area for the 0th feature
census_tracts_ac_utm10.area[0]

In [ ]:
# Get the area for the 0th feature according to its 'ALAND' attribute
census_tracts['ALAND'][0]

In [ ]:
# Check equivalence of the calculated areas and the 'ALAND' column
census_tracts_ac_utm10['ALAND'].values == census_tracts_ac_utm10.area

What explains this disagreement? Are the calculated areas incorrect?

We can also sum the area for Alameda county by adding `.sum()` to the end of our area calculation:

In [ ]:
census_tracts_ac_utm10.area.sum()

We can actually look up how large Alameda County is to check our work. The county is 739 miles<sup>2</sup>, which is around 1,914,001,213 meters<sup>2</sup>. I'd say we're pretty close!

As it turns out, we can similarly use another attribute to get the features' lengths.

**NOTE**: In this case, given we're dealing with polygons, this is equivalent to getting the features' perimeters.

In [ ]:
census_tracts_ac_utm10.length

## Relationship Queries

[Spatial relationship queries](https://en.wikipedia.org/wiki/Spatial_relation) consider how two geometries or sets of geometries relate to one another in space. 

<img src="https://upload.wikimedia.org/wikipedia/commons/5/55/TopologicSpatialRelarions2.png" height="300px"></img>

Here is a list of the most commonly used GeoPandas methods to test spatial relationships:

- [within](http://geopandas.org/reference.html?highlight=distance#geopandas.GeoSeries.within)
- [contains](http://geopandas.org/reference.html?highlight=distance#geopandas.GeoSeries.contains) (the inverse of `within`)
- [intersects](http://geopandas.org/reference.html?highlight=distance#geopandas.GeoSeries.intersects)

There several other GeoPandas spatial relationship predicates, but they are more complex to properly employ. For example, the following two operations only work with geometries that are completely aligned.

- [touches](http://geopandas.org/reference.html?highlight=distance#geopandas.GeoSeries.touches)
- [equals](http://geopandas.org/reference.html?highlight=distance#geopandas.GeoSeries.equals)

All of these methods takes the form:

    Geoseries.<predicate>(geometry)
    
For example:

    Geoseries.contains(geometry)

Let's load a new dataset to demonstrate these queries.

This is a dataset containing all the protected areas (parks and the like) in California.

In [ ]:
parks = gpd.read_file('../data/protected_areas/CPAD_2020a_Units.shp')

Does this need to be reprojected too?

In [ ]:
parks.crs

Yes it does!

Let's reproject it.

In [ ]:
parks_utm10 = parks.to_crs("epsg:26910")

One common use for spatial queries is for spatial subsetting of data.

In our case, let's use `intersects` to find all of the parks that have land in Alameda County.

But before we do that, let's take another look at our geometries.

In [ ]:
census_tracts_ac_utm10.geometry.type.unique()

In [ ]:
census_tracts_ac_utm10.plot()

Because we nave Census tracts, each of these rows is either a Polygon or a MultiPolygon. For our relationship query, we can actually simplify our geometry to be one polygon by using `unary_union`:

In [ ]:
census_tracts_ac_utm10.geometry.unary_union

In [ ]:
print(census_tracts_ac_utm10.geometry.unary_union)

Now, we can go ahead and conduct our operation `intersects`:

In [ ]:
parks_in_ac = parks_utm10.intersects(census_tracts_ac_utm10.geometry.unary_union)

If we scroll the resulting GeoDataFrame to the right, we'll see that the `COUNTY` column of our resulting subset gives us a good confidence check on our results.

In [ ]:
parks_in_ac

In [ ]:
parks_utm10[parks_in_ac].head()

So does this overlay plot!

In [ ]:
# Plot Census tracts
ax = census_tracts_ac_utm10.plot(color='gray', figsize=(12, 16))
# Plot parks
parks_utm10[parks_in_ac].plot(ax=ax,
                              column='ACRES',
                              cmap='summer',
                              legend=True,
                              edgecolor='black',
                              linewidth=0.4, 
                              alpha=0.8,
                              legend_kwds={'label': "acres", 'orientation': "horizontal"})
ax.set_title('Protected areas in Alameda County, colored by area', size=18)

---

### Challenge 1: Spatial Relationship Query

Let's use a spatial relationship query to create a new dataset containing Berkeley schools!

Run the next two cells to load datasets containing Berkeley's city boundary and Alameda County's
schools and to reproject them to EPSG: 26910.

Then in the following cell, write your own code to:

1. Subset the schools for only those `within` Berkeley.
2. Plot the Berkeley boundary and then the schools as an overlay map.

---

In [ ]:
# Load the Berkeley boundary
berkeley = gpd.read_file("../data/berkeley/BerkeleyCityLimits.shp")
# Transform to EPSG:26910
berkeley_utm10 = berkeley.to_crs("epsg:26910")
# Look at GeoDataFrame
berkeley_utm10.head()

In [ ]:
# Load the Alameda County schools CSV
schools_df = pd.read_csv('../data/alco_schools.csv')
# Convert it to a GeoDataFrame
schools_gdf = gpd.GeoDataFrame(schools_df, 
                               geometry=gpd.points_from_xy(schools_df.X, schools_df.Y))
# Define its unprojected (EPSG:4326) CRS
schools_gdf.crs = "epsg:4326"
# Transform to EPSG:26910
schools_gdf_utm10 = schools_gdf.to_crs("epsg:26910")
# Look at GeoDataFrame
schools_df.head()

In [ ]:
# YOUR CODE HERE


## Proximity Analysis

Now that we've seen the basic idea of spatial measurement and relationship queries, let's take a look at a common analysis that combines those concepts: **promximity analysis**.

Proximity analysis seeks to identify all features in a focal feature set that are within some maximum distance of features in a reference feature set.

A common workflow for this analysis is:

1. Buffer (i.e. add a margin around) the reference dataset, out to the maximum distance.
2. Run a spatial relationship query to find all focal features that intersect (or are within) the buffer.

Let's read in our bike boulevard data again. We'll find out which of our Berkeley schools are within a block's distance (200 m) of the boulevards.

In [ ]:
bike_blvds = gpd.read_file('../data/transportation/BerkeleyBikeBlvds.geojson')
bike_blvds.plot()

Of course, we need to reproject the boulevards to our projected CRS.

In [ ]:
bike_blvds_utm10 = bike_blvds.to_crs("epsg:26910")

Now we can create our 200 meter bike boulevard buffers.

In [ ]:
bike_blvds_utm10.crs

In [ ]:
bike_blvds_buf = bike_blvds_utm10.buffer(distance=200)

In [ ]:
bike_blvds_buf.head()

Now, let's overlay everything.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
# Plot Berkeley city boundary
berkeley_utm10.plot(color='lightgrey', ax=ax)
# Plot buffer
bike_blvds_buf.plot(color='pink', ax=ax, alpha=0.5)
# Plot bicycle boulevards
bike_blvds_utm10.plot(ax=ax)
# Plot Berkeley schools
berkeley_schools.plot(color='purple',ax=ax)

Great! Looks like we're all ready to run our intersection to complete the proximity analysis.

**NOTE**: In order to subset with our buffers, we need to call the `unary_union` attribute of the buffer object. This gives us a single unified polygon, rather than a series of multipolygons representing buffers around each of the points in our multilines.

In [ ]:
schools_near_blvds = berkeley_schools.within(bike_blvds_buf.unary_union)
blvd_schools = berkeley_schools[schools_near_blvds]

Now let's overlay again, to see if the schools we subsetted make sense.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
# Plot Berkeley city boundary
berkeley_utm10.plot(color='lightgrey', ax=ax)
# Plot buffer
bike_blvds_buf.plot(color='pink', ax=ax, alpha=0.5)
# Plot bicycle boulevards
bike_blvds_utm10.plot(ax=ax)
# Plot Berkeley schools
berkeley_schools.plot(color='purple',ax=ax)
# Plot schools within buffer 
blvd_schools.plot(color='yellow', markersize=50, ax=ax)

If we want to find the shortest distance from one school to the bike boulevards, we can use the `distance` function.

In [ ]:
berkeley_schools.distance(bike_blvds_utm10.unary_union)

---

### Challenge 2: Proximity Analysis

Now it's your turn to try out a proximity analysis!

Run the next cell to load BART-system data, reproject it to EPSG: 26910, and subset it to Berkeley.

Then in the following cell, write your own code to find all schools within walking distance (1 km) of a BART station.

As a reminder, let's break this into steps:

1. Buffer your Berkeley BART stations to 1 km (**HINT**: remember your units!).
2. Use the schools' `within` attribute to check whether or not they're within the buffers (**HINT**: don't forget the `unary_union`!).
3. Subset the Berkeley schools using the object returned by your spatial relationship query.
4. As always, plot your results for a good visual check!

---

In [ ]:
# Load the BART stations from CSV
bart_stations = pd.read_csv('../data/transportation/bart.csv')
# Convert to a GeoDataFrame
bart_stations_gdf = gpd.GeoDataFrame(bart_stations, 
                                     geometry=gpd.points_from_xy(bart_stations.lon, bart_stations.lat))
# Define its unprojected (EPSG:4326) CRS
bart_stations_gdf.crs = "epsg:4326"
# Transform to UTM Zone 10 N (EPSG:26910)
bart_stations_gdf_utm10 = bart_stations_gdf.to_crs("epsg:26910")
# Subset to Berkeley
berkeley_bart = bart_stations_gdf_utm10[bart_stations_gdf_utm10.within(berkeley_utm10.unary_union)]

In [ ]:
# YOUR CODE HERE
